# Charts open > X days
This script will:

- This script will:

- Filter visits that occurred more than 48 hours ago.
- Add a “Days Since Visit” column.
- Export a separate Excel file for each doctor (5 total).
- Export a separate Excel file for each clinic (3 total), with data sorted by doctor and then by days since visit (oldest first).
- Display the data for each doctor and each clinic in the terminal output.

In [21]:

import pandas as pd
from datetime import datetime, timedelta, time

def filter_and_export_visits_by_physician_and_clinic(excel_path):
    # Load the Excel file
    df = pd.read_excel(excel_path)
    print(f'Here are the sample data:')
    print(df.head(10))
    
    print(f'---------------------------------')
    # Ensure the date column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Define today's 5 PM timestamp
    now = datetime.now()
    today_5pm = datetime.combine(now.date(), time(17, 0))

    # Calculate cutoff time: 48 hours ago from today's 5 PM
    cutoff_time = today_5pm - timedelta(hours=48)

    # Calculate days since visit
    df['Days Since Visit'] = ((today_5pm - df['Date']).dt.total_seconds() / 86400).round().astype(int)

    # Filter visits older than 48 hours
    df_filtered = df[df['Date'] < cutoff_time]

    # Export a separate file for each doctor
    for physician in df_filtered['Doctor'].unique():
        print(f'\n\nThe following data will be sent in an xlsx file to {physician}')
        df_physician = df_filtered[df_filtered['Doctor'] == physician].sort_values(by='Date')
        filename = f"{physician.replace(' ', '_')}_visits.xlsx"
        df_physician.to_excel(filename, index=False)
        print(f"\nVisits for {physician}:")
        print(df_physician)
    print(f'\n\n\nNow generating XLSX files for all CLINICS')
    # Export a separate file for each clinic, sorted by Doctor and then by Date (oldest first)
    for clinic in df_filtered['Clinic'].unique():
        print(f'\n\nThe following data will be sent in an xlsx file to {clinic}')
        df_clinic = df_filtered[df_filtered['Clinic'] == clinic]
        df_clinic_sorted = df_clinic.sort_values(by=['Doctor', 'Date'])
        filename = f"{clinic.replace(' ', '_')}_clinic_visits.xlsx"
        df_clinic_sorted.to_excel(filename, index=False)
        print(f"\nVisits for {clinic}:")
        print(df_clinic_sorted)
        
# Sample usage
filtered_df = filter_and_export_visits_by_physician_and_clinic("encounter_data_recent_2_weeks.xlsx")
print(filtered_df)


Here are the sample data:
        Doctor     Name       Date    Clinic
0  Dr. Johnson    Julia 2025-05-28  Clinic A
1  Dr. Johnson   George 2025-05-23  Clinic A
2  Dr. Johnson    Julia 2025-05-28  Clinic A
3   Dr. Garcia  Charlie 2025-05-22  Clinic C
4    Dr. Smith    Diana 2025-05-29  Clinic A
5   Dr. Garcia  Charlie 2025-05-30  Clinic C
6   Dr. Garcia    Diana 2025-05-29  Clinic C
7   Dr. Garcia    Alice 2025-05-29  Clinic C
8   Dr. Garcia      Ian 2025-05-27  Clinic C
9    Dr. Smith    Fiona 2025-05-27  Clinic A
---------------------------------


The following data will be sent in an xlsx file to Dr. Johnson

Visits for Dr. Johnson:
         Doctor     Name       Date    Clinic  Days Since Visit
15  Dr. Johnson    Fiona 2025-05-20  Clinic A                11
30  Dr. Johnson  Charlie 2025-05-20  Clinic A                11
13  Dr. Johnson    Julia 2025-05-21  Clinic A                10
46  Dr. Johnson      Ian 2025-05-21  Clinic A                10
10  Dr. Johnson    Ethan 2025-05-22